# Initialisation

In [ ]:
%matplotlib ipympl
from matplotlib import pyplot as plt
import numpy as np
#import scipy
from scipy.special import zeta
from time import time
from astropy import cosmology as cosmo
from astropy import units as u
from astropy import constants as c
from astropy.visualization import quantity_support
quantity_support()

# Models

## Planck18

In [ ]:
Planck18 = cosmo.Planck18

eta = 6e-10
Y = .24
neutrinos = (7/8)*Planck18.Neff*(4/11)**(4/3)

z_p18 = np.logspace(-2, 4, 301)

t_p18 = Planck18.age(z_p18).to_value(u.yr)
Or_p18 = Planck18.Ogamma(z_p18) * (1+neutrinos)
Ob_p18 = Planck18.Ob(z_p18)
Om_p18 = Planck18.Om(z_p18)
ODE_p18 = Planck18.Ode(z_p18)

n_H_p18 = (1-Y) * Ob_p18*Planck18.critical_density(z_p18) / c.m_p
y_p18 = (2/n_H_p18) * np.power(2*np.pi*c.m_e*c.k_B*Planck18.Tcmb(z_p18)/c.h**2, 1.5) * np.exp(-13.6*u.eV/c.k_B/Planck18.Tcmb(z_p18))
x0 = np.linspace(1e-12, 1-1e-12, 1001)
y0 = x0**2/(1-x0)
x_p18 = np.interp(y_p18, y0, x0)
t_coll_p18 = (1/x_p18/n_H_p18/c.sigma_T/c.c).to_value(u.yr)
t_cmb_p18 = np.min(t_p18[t_coll_p18 > t_p18])

rho_c_p18 = Planck18.critical_density(z_p18).to_value(u.Msun/u.Mpc**3)


## Coasting

Parameters:

In [ ]:
eta = 6e-10
#eta = 6.13e-10

H0 = 70 * u.km/u.s/u.Mpc
#H0 = Planck18.H0

Ok = -.05
print('t0', (1/H0).to_value(u.Gyr), 'Gyr')

# photons
rho_gamma0 = 4*c.sigma_sb * Planck18.Tcmb0**4 / c.c**3
Ogamma0 = (rho_gamma0 * 8*np.pi*c.G / 3/H0**2).to_value(u.dimensionless_unscaled)

# masless neutrinos
neutrinos = (7/8)*Planck18.Neff*(4/11)**(4/3)
Or0 = Ogamma0 * (1 + neutrinos)

# (non-relativistic) matter
mu0 = ((1 - Ok) / Or0 - 3) / 2
t0_teq = np.sqrt(5*mu0**4 / (3+2*mu0))
print('Omega_r', Or0, Planck18.Ogamma0*(1+neutrinos))

print('mu, zeq', mu0, t0_teq - 1, Planck18.Om0/Planck18.Ogamma0)

Om0 = mu0 * Or0
rho_m0 = Om0 * 3*H0**2/(8*np.pi*c.G)
print('Omega_m', Om0, mu0 / (3 + 2 * mu0) * (1-Ok))

n_gamma0 = 16*np.pi*zeta(3) * (c.k_B*Planck18.Tcmb0 / c.h/c.c)**3
rho_b0 = eta * c.m_p * n_gamma0
Ob0 = (rho_b0 * 8*np.pi*c.G / 3/H0**2).to_value(u.dimensionless_unscaled)
print('Omega_b', Ob0, Planck18.Ob0)

#eta_m = (rho_m0 / rho_r0).to_value(u.dimensionless_unscaled)
#print(eta_m)

Evolution:

In [ ]:
mu = np.logspace(-4, 0, 501) * mu0

t_teq = np.sqrt(5*mu**4 / (3+2*mu))

z_model = t0_teq / t_teq - 1
t_model = (1/H0 / (1+z_model)).to_value(u.yr)

Or_model = (1 - Ok) / (3 + 2 * mu)
Om_model = (1 - Ok) * mu / (3 + 2 * mu)
Ob_model = Ob0/Om0 * Om_model
ODE_model = 2*Or_model + Om_model

rho_c_model = 3 / 8/np.pi/c.G / (t_model*u.yr)**2
Tcmb_model = np.power(c.c**3/(4*c.sigma_sb) * Or_model * rho_c_model / (1+neutrinos), 1/4)

n_H_model = (1-Y) * Ob_model*rho_c_model / c.m_p
y_model = (2/n_H_model) * np.power(2*np.pi*c.m_e*c.k_B*Tcmb_model/c.h**2, 1.5) * np.exp(-13.6*u.eV/c.k_B/Tcmb_model)
x0 = np.linspace(0, 1-1e-6, 1001)
y0 = x0**2/(1-x0)
x_model = np.interp(y_model, y0, x0)
t_coll_model = (1/x_model/n_H_model/c.sigma_T/c.c).to_value(u.yr)
t_cmb_model = np.min(t_model[t_coll_model > t_model])
z_cmb_model = (1/H0).to_value(u.yr) / t_cmb_model

rho_c_model = rho_c_model.to_value(u.Msun/u.Mpc**3)


# Plots

## Cosmic evolution

In [ ]:
fig_name = 'evolution'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(8, 8))
axes = fig.subplots(nrows=2, ncols=1, squeeze=False,
                    sharex='col', sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )
alpha = 1

ax = axes[0, 0]
ax.set_ylabel(r'density $\rho$ [M$_\odot$ Mpc$^{-3}$]')

def mass_to_E(m):
    return((m*c.c**2 * u.Msun/u.Mpc**3).to_value(u.GeV/u.cm**3))
def E_to_mass(E):
    return((E/c.c**2 * u.GeV/u.cm**3).to_value(u.Msun/u.Mpc**3))
secax = ax.secondary_yaxis('right', functions=(mass_to_E, E_to_mass))
secax.set_ylabel(r'energy density $\rho c^2$ [GeV cm$^{-3}$]')


ax.set_yscale('log')
ax.set_xscale('log')

l1, = ax.plot(t_p18, ODE_p18*rho_c_p18, 'r--', alpha=alpha, label='dark energy')
l2, = ax.plot(t_p18, (Om_p18-Ob_p18)*rho_c_p18, 'k--', alpha=alpha, label='dark matter')
l3, = ax.plot(t_p18, Ob_p18*rho_c_p18, 'c--', alpha=alpha, label='baryons (atoms)')
l4, = ax.plot(t_p18, Or_p18*rho_c_p18/(1+neutrinos), 'y--', alpha=alpha, label='photons')
first_legend = ax.legend(handles=[l1, l2, l3, l4], title=r'$\Lambda$CDM', loc='upper left', bbox_to_anchor=(0.45, 0.5, 0.5, 0.5))
ax.add_artist(first_legend)

l1, = ax.plot(t_model, ODE_model*rho_c_model, 'r-', alpha=alpha, label='dark energy')
l2, = ax.plot(t_model, (Om_model-Ob_model)*rho_c_model, 'k-', alpha=alpha, label='dark matter')
l3, = ax.plot(t_model, Ob_model*rho_c_model, 'c-', alpha=alpha, label='baryons (atoms)')
l4, = ax.plot(t_model, Or_model*rho_c_model/(1+neutrinos), 'y-', alpha=alpha, label='photons')
ax.legend(handles=[l1, l2, l3, l4], title='This work', loc='upper right', bbox_to_anchor=(0.5, 0.5, 0.5, 0.5))


ax = axes[1, 0]
ax.set_ylabel('ionisation fraction $x$')

ax.plot(t_p18, x_p18, 'r--', alpha=alpha, label=r'$\Lambda$CDM: $t_{\rm CMB} \sim$'+f'{t_cmb_p18:g} yr')
ax.plot(t_model, x_model, 'k-', alpha=alpha, label=r'This work: $t_{\rm CMB} \sim$'+f'{t_cmb_model:g} yr')

ax.scatter(t_cmb_p18, np.interp(t_cmb_p18, t_p18[::-1], x_p18[::-1]), marker='d', facecolors='none', edgecolors='r')
ax.plot(t_cmb_model, np.interp(t_cmb_model, t_model, x_model), 'kd')

ax.legend()

ax.set_xscale('log')
ax.set_xlabel('cosmic time $t$ [yr]')


for ax in axes.flatten():
    ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
    ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
    ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
    ax.grid(True, which='both')
    #ax.axvline(t_cmb_p18, c='r', ls='--')
    #ax.axvline(t_cmb_model, c='k', ls='-')
fig.set_tight_layout(True)
plt.show()
plt.savefig('figures/evolution.pdf')

## Old plots

### Energy density

In [ ]:
fig_name = 'density_comparison'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(8, 5))
axes = fig.subplots(nrows=1, ncols=1, squeeze=False,
                    sharex='col', sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )
alpha = 1

ax = axes[0, 0]
#ax.set_ylabel(r'density $\rho$')
ax.set_ylabel(r'density $\rho$ [M$_\odot$ Mpc$^{-3}$]')

def mass_to_E(m):
    return((m*c.c**2 * u.Msun/u.Mpc**3).to_value(u.GeV/u.cm**3))
def E_to_mass(E):
    return((E/c.c**2 * u.GeV/u.cm**3).to_value(u.Msun/u.Mpc**3))
secax = ax.secondary_yaxis('right', functions=(mass_to_E, E_to_mass))
secax.set_ylabel(r'energy density $\rho c^2$ [GeV cm$^{-3}$]')


#ax.set_ylabel(r'density $\rho(t) / \rho_c(t_0) = (8\pi G\rho)/(3H_0^2)$')
#ax.set_ylabel(r'$\rho(t) / \rho_c(t_0) = (8\pi G\rho)/(3H_0^2)$')
#ax.set_ylim(3e-5, 3e-1)
#ax.set_ylim(3e-5, 3e12)
ax.set_yscale('log')
ax.set_xscale('log')

'''
'''
l1, = ax.plot(t_p18, ODE_p18*rho_c_p18, 'r--', alpha=alpha, label='dark energy')
l2, = ax.plot(t_p18, (Om_p18-Ob_p18)*rho_c_p18, 'k--', alpha=alpha, label='dark matter')
l3, = ax.plot(t_p18, Ob_p18*rho_c_p18, 'c--', alpha=alpha, label='baryons (atoms)')
l4, = ax.plot(t_p18, Or_p18*rho_c_p18/(1+neutrinos), 'y--', alpha=alpha, label='photons')
first_legend = ax.legend(handles=[l1, l2, l3, l4], title=r'$\Lambda$CDM', loc='upper left', bbox_to_anchor=(0.45, 0.5, 0.5, 0.5))
ax.add_artist(first_legend)

l1, = ax.plot(t_model, ODE_model*rho_c_model, 'r-', alpha=alpha, label='dark energy')
l2, = ax.plot(t_model, (Om_model-Ob_model)*rho_c_model, 'k-', alpha=alpha, label='dark matter')
l3, = ax.plot(t_model, Ob_model*rho_c_model, 'c-', alpha=alpha, label='baryons (atoms)')
l4, = ax.plot(t_model, Or_model*rho_c_model/(1+neutrinos), 'y-', alpha=alpha, label='photons')
ax.legend(handles=[l1, l2, l3, l4], title='This work', loc='upper right', bbox_to_anchor=(0.5, 0.5, 0.5, 0.5))

'''
ax.plot(z_p18, Planck18.Tcmb(z_p18), 'r--', alpha=alpha, label=r'$\Lambda$CDM radiation')
#ax.plot(z_p18, Tcmb_p18, 'r-', alpha=alpha, label=r'$\Lambda$CDM radiation')
ax.plot(z_model, Tcmb_model, 'r-', alpha=alpha, label='coasting radiation')
#
ax.plot(z_p18, Or_p18, 'r--', alpha=alpha, label=r'$\Lambda$CDM radiation')
ax.plot(z_model, Or_model, 'r-', alpha=alpha, label='coasting radiation')
'''


#fig.legend(ncol=2, title=['a', 'n'])


ax.set_xscale('log')
#ax.set_xlim(3e-1, 3e3)
#ax.set_xlim(3e4, 3e10)
ax.set_xlabel('t [yr]')

for ax in axes.flatten():
    ax.tick_params(which='both', direction='in')
    ax.grid(alpha=.5)
fig.set_tight_layout(True)
plt.show()
#plt.savefig('density.pdf')

### CMB

In [ ]:
fig_name = 'CMB'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(8, 5))
axes = fig.subplots(nrows=1, ncols=1, squeeze=False,
                    sharex='col', sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )

alpha = 1

ax = axes[0, 0]
ax.set_ylabel('ionisation fraction $x$')

ax.plot(t_p18, x_p18, 'r--', alpha=alpha, label=r'$\Lambda$CDM: $t_{\rm CMB} \sim$'+f'{t_cmb_p18:g} yr')
ax.plot(t_model, x_model, 'k-', alpha=alpha, label=r'This work: $t_{\rm CMB} \sim$'+f'{t_cmb_model:g} yr')

ax.scatter(t_cmb_p18, np.interp(t_cmb_p18, t_p18[::-1], x_p18[::-1]), marker='d', facecolors='none', edgecolors='r')
ax.plot(t_cmb_model, np.interp(t_cmb_model, t_model, x_model), 'kd')

ax.legend()

ax.set_xscale('log')
#ax.set_xlim(3e4, 3e10)
ax.set_xlabel('cosmic time $t$ [yr]')

for ax in axes.flatten():
    ax.tick_params(which='both', direction='in')
    ax.grid(alpha=.5)
fig.set_tight_layout(True)
plt.show()
#plt.savefig('CMB.pdf')

In [ ]:
fig_name = 'CMB_all'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(5, 10))
axes = fig.subplots(nrows=4, ncols=1, squeeze=False,
                    sharex='col', sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )

alpha = 1

ax = axes[0, 0]
ax.set_ylabel(r'radiation temperature $T$ [K]')
ax.set_ylim(.3, 3e5)
ax.set_yscale('log')

ax.plot(t_p18, Planck18.Tcmb(z_p18), 'r--', alpha=alpha, label=r'$\Lambda$CDM')
ax.plot(t_model, Tcmb_model, 'k-', alpha=alpha, label='coasting')

ax = axes[1, 0]
ax.set_ylabel('hydrogen density $n_H$ [cm$^{-3}$]')
ax.set_yscale('log')
#ax.set_ylim(3e-2, 3e10)

ax.plot(t_p18, n_H_p18.to_value(u.cm**-3), 'r--', alpha=alpha, label=r'$\Lambda$CDM')
ax.plot(t_model, n_H_model.to_value(u.cm**-3), 'k-', alpha=alpha, label='coasting')


ax = axes[2, 0]
ax.set_ylabel('ionisation fraction $x$')
#ax.set_yscale('log')
#ax.set_ylim(3e-2, 3e10)

ax.plot(t_p18, x_p18, 'r--', alpha=alpha, label=r'$\Lambda$CDM')
ax.plot(t_model, x_model, 'k-', alpha=alpha, label='coasting')


ax = axes[3, 0]
ax.set_ylabel('collision timescale $t_{coll}$ [yr]')
ax.set_yscale('log')
ax.set_ylim(3e-2, 3e11)

ax.plot(t_p18, (1/x_p18/n_H_p18/c.sigma_T/c.c).to_value(u.yr), 'r--', alpha=alpha, label=r'$\Lambda$CDM')
ax.plot(t_model, (1/x_model/n_H_model/c.sigma_T/c.c).to_value(u.yr), 'k-', alpha=alpha, label='coasting')


ax.set_xscale('log')
#ax.set_xlim(3e4, 3e10)
ax.set_xlabel('cosmic time $t$ [yr]')

for ax in axes.flatten():
    ax.tick_params(which='both', direction='in')
    ax.grid(alpha=.5)
    ax.axvline(t_cmb_p18, c='r', ls='--')
    ax.axvline(t_cmb_model, c='k', ls='-')
fig.set_tight_layout(True)
plt.show()